<a href="https://colab.research.google.com/github/aymanmarr/TestCodeEnsa/blob/main/MicroMouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install Pillow numpy

In [5]:
"""
RÉSOLVEUR DE LABYRINTHE - ALGORITHME A* AVEC GIF ANIMÉ
Détecte l'entrée par analyse des variations de couleur/épaisseur sur les bords
"""

from collections import deque
import heapq
from PIL import Image, ImageDraw
import numpy as np
import time

class MazeAStar:
    # CORRIGÉ: __init__ avec deux underscores
    def __init__(self, image_path):
        """Charger et analyser l'image du labyrinthe"""
        print("📂 Chargement de l'image...")
        self.image_path = image_path
        self.original_image = Image.open(image_path)

        # Garder l'image en niveaux de gris pour l'analyse
        self.gray_image = self.original_image.convert('L')
        self.gray_array = np.array(self.gray_image)

        # Image binaire pour la navigation
        self.maze = self._process_image()
        self.height = len(self.maze)
        self.width = len(self.maze[0])
        print(f"✅ Labyrinthe chargé: {self.width}x{self.height} pixels")

        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    def _process_image(self):
        """Convertir l'image en matrice binaire"""
        # < 128 signifie que le noir (0) est un mur (1)
        # et le blanc (255) est un chemin (0)
        maze = (self.gray_array < 128).astype(int)
        return maze

    def is_valid(self, x, y):
        """Vérifier si la position est valide"""
        return (0 <= x < self.width and
                0 <= y < self.height and
                self.maze[y][x] == 0) # 0 = chemin

    def detect_entry_by_border_analysis(self):
        """
        Détecter l'entrée en analysant les variations de couleur/épaisseur
        sur les bords du labyrinthe
        """
        print("\n🔍 Analyse des bords pour détecter l'entrée...")

        border_thickness = 20  # Épaisseur de la zone à analyser

        # Analyser chaque bord
        regions = []

        # Bord GAUCHE
        left_region = self.gray_array[:, :border_thickness]
        left_mean = np.mean(left_region)
        left_std = np.std(left_region)
        regions.append(('gauche', left_mean, left_std, left_region))

        # Bord DROIT
        right_region = self.gray_array[:, -border_thickness:]
        right_mean = np.mean(right_region)
        right_std = np.std(right_region)
        regions.append(('droite', right_mean, right_std, right_region))

        # Bord HAUT
        top_region = self.gray_array[:border_thickness, :]
        top_mean = np.mean(top_region)
        top_std = np.std(top_region)
        regions.append(('haut', top_mean, top_std, top_region))

        # Bord BAS
        bottom_region = self.gray_array[-border_thickness:, :]
        bottom_mean = np.mean(bottom_region)
        bottom_std = np.std(bottom_region)
        regions.append(('bas', bottom_mean, bottom_std, bottom_region))

        print("\n📊 Analyse des bords:")
        for side, mean, std, _ in regions:
            print(f"    {side:8} - Moyenne: {mean:.1f}, Écart-type: {std:.1f}")

        # Trouver le bord avec la plus grande variation (std) ou la plus grande luminosité
        # Cela indique souvent une zone différente/marquée
        max_std_region = max(regions, key=lambda r: r[2])
        max_mean_region = max(regions, key=lambda r: r[1])

        print(f"\n🎯 Bord avec plus de variation: {max_std_region[0]}")
        print(f"🎯 Bord le plus clair: {max_mean_region[0]}")

        # Choisir le bord avec le plus de variations comme candidat
        candidate_side = max_std_region[0]

        # Trouver un point d'entrée sur ce bord
        entry = self._find_entry_on_side(candidate_side)

        if entry:
            print(f"✅ ENTRÉE détectée: {entry} sur le bord {candidate_side}")
            return entry

        # Sinon, essayer le bord le plus clair
        print(f"⚠️  Tentative sur le bord le plus clair: {max_mean_region[0]}")
        entry = self._find_entry_on_side(max_mean_region[0])

        if entry:
            print(f"✅ ENTRÉE détectée: {entry} sur le bord {max_mean_region[0]}")
            return entry

        # En dernier recours, scanner tous les bords pour trouver un passage
        print("⚠️  Recherche exhaustive sur tous les bords...")
        return self._find_any_border_passage()

    def _find_entry_on_side(self, side):
        """Trouver un point d'entrée valide sur un côté spécifique"""
        search_depth = 30  # Profondeur de recherche depuis le bord

        if side == 'gauche':
            # Chercher du bas vers le haut sur le bord gauche
            for y in range(self.height - 1, -1, -1):
                for x in range(search_depth):
                    if self.is_valid(x, y):
                        return (x, y)

        elif side == 'droite':
            for y in range(self.height):
                for x in range(self.width - 1, self.width - search_depth, -1):
                    if self.is_valid(x, y):
                        return (x, y)

        elif side == 'haut':
            for x in range(self.width):
                for y in range(search_depth):
                    if self.is_valid(x, y):
                        return (x, y)

        elif side == 'bas':
            for x in range(self.width):
                for y in range(self.height - 1, self.height - search_depth, -1):
                    if self.is_valid(x, y):
                        return (x, y)

        return None

    def _find_any_border_passage(self):
        """Recherche exhaustive d'un passage sur n'importe quel bord"""
        search_depth = 40

        # Priorité: gauche-bas, gauche, bas, droite, haut

        # Gauche-bas (coin)
        for y in range(self.height - 1, max(0, self.height - search_depth), -1):
            for x in range(search_depth):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - coin gauche-bas")
                    return (x, y)

        # Bord gauche complet
        for y in range(self.height):
            for x in range(search_depth):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - bord gauche")
                    return (x, y)

        # Bord bas
        for x in range(self.width):
            for y in range(self.height - 1, self.height - search_depth, -1):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - bord bas")
                    return (x, y)

        # Bord droit
        for y in range(self.height):
            for x in range(self.width - 1, self.width - search_depth, -1):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - bord droit")
                    return (x, y)

        # Bord haut
        for x in range(self.width):
            for y in range(search_depth):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - bord haut")
                    return (x, y)

        raise Exception("❌ Aucun passage trouvé sur les bords!")

    def find_center(self):
        """Trouver le centre accessible du labyrinthe"""
        print("\n🎯 Recherche du centre du labyrinthe...")

        center_x = self.width // 2
        center_y = self.height // 2

        # Chercher en spirale autour du centre géométrique
        max_radius = max(self.width, self.height) // 2

        for radius in range(0, max_radius, 2):
            for angle in range(0, 360, 15):
                rad = np.radians(angle)
                dx = int(radius * np.cos(rad))
                dy = int(radius * np.sin(rad))

                x = center_x + dx
                y = center_y + dy

                if (0 <= x < self.width and
                    0 <= y < self.height and
                    self.maze[y][x] == 0):
                    print(f"✅ Centre trouvé: ({x}, {y})")
                    return (x, y)

        raise Exception("❌ Aucun centre accessible trouvé!")

    def find_entry_and_exit(self):
        """
        Trouver automatiquement:
        - ENTRÉE: zone différente sur le bord (analyse de couleur/épaisseur)
        - SORTIE: centre du labyrinthe
        """
        print("\n" + "="*60)
        print("🎯 DÉTECTION AUTOMATIQUE ENTRÉE/SORTIE")
        print("="*60)

        entry = self.detect_entry_by_border_analysis()
        exit_center = self.find_center()

        distance = abs(exit_center[0] - entry[0]) + abs(exit_center[1] - entry[1])
        print(f"\n📏 Distance Manhattan: {distance} pixels")
        print("="*60)

        return entry, exit_center

    def heuristic(self, point, goal):
        """Distance de Manhattan"""
        return abs(point[0] - goal[0]) + abs(point[1] - goal[1])

    def a_star(self, start, goal):
        """Algorithme A*"""
        print(f"\n⚡ Application de l'algorithme A*...")
        print(f"    Départ (ENTRÉE): {start}")
        print(f"    Arrivée (CENTRE): {goal}")

        start_time = time.time()

        counter = 0
        open_set = []
        heapq.heappush(open_set, (0, counter, start))

        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}

        closed_set = set()
        nodes_explored = 0

        while open_set:
            _, _, current = heapq.heappop(open_set)

            if current == goal:
                end_time = time.time()
                path = self._reconstruct_path(came_from, goal)

                stats = {
                    'length': len(path),
                    'time': end_time - start_time,
                    'nodes_explored': nodes_explored,
                    'success': True
                }

                print(f"✅ Chemin trouvé!")
                print(f"    📏 Longueur: {len(path)} pas")
                print(f"    ⏱  Temps: {stats['time']:.4f} secondes")
                print(f"    🔢 Nœuds explorés: {nodes_explored}")

                return path, stats

            if current in closed_set:
                continue

            closed_set.add(current)
            nodes_explored += 1
            x, y = current

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy
                neighbor = (nx, ny)

                if not self.is_valid(nx, ny) or neighbor in closed_set:
                    continue

                tentative_g = g_score[current] + 1

                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, goal)

                    counter += 1
                    heapq.heappush(open_set, (f_score[neighbor], counter, neighbor))

        end_time = time.time()
        stats = {
            'length': -1,
            'time': end_time - start_time,
            'nodes_explored': nodes_explored,
            'success': False
        }

        print(f"❌ Aucun chemin trouvé!")
        return None, stats

    def _reconstruct_path(self, came_from, current):
        """Reconstruire le chemin"""
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

    def create_animated_gif(self, path, start, goal, output_path='solution.gif',
                            frame_skip=5, duration=50, final_pause=2000):
        """Créer un GIF animé montrant la progression du chemin"""
        print(f"\n🎬 Génération du GIF animé...")

        frames = []

        path_length = len(path)
        frame_indices = list(range(0, path_length, frame_skip))

        if frame_indices[-1] != path_length - 1:
            frame_indices.append(path_length - 1)

        print(f"    📊 {len(frame_indices)} frames à générer...")

        for i, end_idx in enumerate(frame_indices):
            frame_img = self.original_image.convert('RGB')
            draw = ImageDraw.Draw(frame_img)

            current_path = path[:end_idx + 1]

            if len(current_path) > 1:
                for j in range(len(current_path) - 1):
                    x1, y1 = current_path[j]
                    x2, y2 = current_path[j + 1]
                    draw.line([(x1, y1), (x2, y2)], fill=(255, 0, 0), width=3)

            if current_path:
                cx, cy = current_path[-1]
                r = 6
                draw.ellipse([(cx-r, cy-r), (cx+r, cy+r)],
                             fill=(255, 255, 0), outline=(255, 255, 0))

            sx, sy = start
            r = 8
            draw.ellipse([(sx-r, sy-r), (sx+r, sy+r)],
                         fill=(0, 255, 0), outline=(0, 255, 0))

            gx, gy = goal
            draw.ellipse([(gx-r, gy-r), (gx+r, gy+r)],
                         fill=(0, 0, 255), outline=(0, 0, 255))

            frames.append(frame_img)

            if (i + 1) % 10 == 0:
                print(f"    ⏳ Progression: {i+1}/{len(frame_indices)} frames")

        print(f"    💾 Sauvegarde du GIF...")

        durations = [duration] * (len(frames) - 1) + [final_pause]

        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=durations,
            loop=0,
            optimize=False
        )

        print(f"✅ GIF animé sauvegardé: {output_path}")
        print(f"    🎞  Frames: {len(frames)}")
        print(f"    🟢 Vert = ENTRÉE (bord différent)")
        print(f"    🔴 Rouge = Chemin parcouru")
        print(f"    🟡 Jaune = Position actuelle")
        print(f"    🔵 Bleu = SORTIE (centre)")

    def visualize_solution(self, path, start, goal, output_path='solution.png'):
        """Créer une image statique avec le chemin tracé"""
        print(f"\n🎨 Génération de l'image statique...")

        result_img = self.original_image.convert('RGB')
        draw = ImageDraw.Draw(result_img)

        if path and len(path) > 1:
            for i in range(len(path) - 1):
                x1, y1 = path[i]
                x2, y2 = path[i + 1]
                draw.line([(x1, y1), (x2, y2)], fill=(255, 0, 0), width=3)

            sx, sy = start
            r = 8
            draw.ellipse([(sx-r, sy-r), (sx+r, sy+r)],
                         fill=(0, 255, 0), outline=(0, 255, 0))

            gx, gy = goal
            draw.ellipse([(gx-r, gy-r), (gx+r, gy+r)],
                         fill=(0, 0, 255), outline=(0, 0, 255))

        result_img.save(output_path)
        print(f"✅ Image statique sauvegardée: {output_path}")

        return result_img

    def solve_auto(self, output_gif='solution.gif', output_png='solution.png',
                   frame_skip=5, duration=50):
        """Résolution automatique: entrée (bord différent) → sortie (centre)"""
        print("\n" + "="*60)
        print("🤖 RÉSOLUTION AUTOMATIQUE DU LABYRINTHE")
        print("="*60)

        try:
            # Détection automatique
            entry, center = self.find_entry_and_exit()

            path, stats = self.a_star(entry, center)

            if stats['success']:
                self.visualize_solution(path, entry, center, output_png)
                self.create_animated_gif(path, entry, center, output_gif,
                                         frame_skip=frame_skip, duration=duration)

                print("\n" + "="*60)
                print("✅ RÉSOLUTION RÉUSSIE!")
                print("="*60)
                print(f"📊 STATISTIQUES:")
                print(f"    • Longueur du chemin: {stats['length']} pas")
                print(f"    • Temps de calcul: {stats['time']:.4f} secondes")
                print(f"    • Nœuds explorés: {stats['nodes_explored']}")
                print("="*60)

                return path, stats
            else:
                print("\n❌ Impossible de trouver un chemin!")
                return None, stats

        except Exception as e:
            print(f"\n❌ Erreur: {e}")
            import traceback
            traceback.print_exc()
            return None, None


def main():
    """Programme principal"""

    image_path = 'maze.jpg'

    print("🚀 Démarrage du résolveur de labyrinthe...")
    print(f"📁 Fichier: {image_path}\n")

    try:
        solver = MazeAStar(image_path)

        # Résolution automatique avec détection intelligente
        path, stats = solver.solve_auto(
            output_gif='solution.gif',
            output_png='solution.png',
            frame_skip=5, # Augmenter pour un GIF plus rapide (ex: 15)
            duration=50  # Durée de chaque frame en ms
        )

        if path:
            print("\n✨ Fichiers générés:")
            print("    📄 solution.png - Image statique du chemin complet")
            print("    🎬 solution.gif - Animation de la progression")
            print("\n💡 ENTRÉE (vert) = Zone différente sur le bord")
            print("    SORTIE (bleu) = Centre du labyrinthe")

    except FileNotFoundError:
        print(f"\n❌ ERREUR: Le fichier '{image_path}' n'existe pas!")
        print("📝 Placez votre image 'labyrinthe.png' dans le dossier du script")

    except Exception as e:
        print(f"\n❌ ERREUR: {e}")
        import traceback
        traceback.print_exc()


# CORRIGÉ: __name__ et __main__ avec deux underscores
if __name__ == "__main__":
    main()

🚀 Démarrage du résolveur de labyrinthe...
📁 Fichier: maze.jpg

📂 Chargement de l'image...
✅ Labyrinthe chargé: 426x432 pixels

🤖 RÉSOLUTION AUTOMATIQUE DU LABYRINTHE

🎯 DÉTECTION AUTOMATIQUE ENTRÉE/SORTIE

🔍 Analyse des bords pour détecter l'entrée...

📊 Analyse des bords:
    gauche   - Moyenne: 185.8, Écart-type: 113.1
    droite   - Moyenne: 161.0, Écart-type: 122.7
    haut     - Moyenne: 173.7, Écart-type: 118.6
    bas      - Moyenne: 158.7, Écart-type: 123.3

🎯 Bord avec plus de variation: bas
🎯 Bord le plus clair: gauche
✅ ENTRÉE détectée: (5, 424) sur le bord bas

🎯 Recherche du centre du labyrinthe...
✅ Centre trouvé: (213, 216)

📏 Distance Manhattan: 416 pixels

⚡ Application de l'algorithme A*...
    Départ (ENTRÉE): (5, 424)
    Arrivée (CENTRE): (213, 216)
✅ Chemin trouvé!
    📏 Longueur: 2315 pas
    ⏱  Temps: 1.0811 secondes
    🔢 Nœuds explorés: 111509

🎨 Génération de l'image statique...
✅ Image statique sauvegardée: solution.png

🎬 Génération du GIF animé...
    📊 46

In [11]:
"""
RÉSOLVEUR DE LABYRINTHE - ALGORITHME A* CENTRÉ (AVEC DISTANCE TRANSFORM)
Version complète - Détection auto entrée/sortie et chemin centré.
"""

import heapq
from PIL import Image, ImageDraw
import numpy as np
import time
import traceback
# [NÉCESSAIRE] Importation pour la carte des distances
from scipy.ndimage import distance_transform_edt

class MazeAStar:

    # [MODIFIÉ]
    def __init__(self, image_path):
        """Charger, analyser l'image et créer la carte des distances"""
        print("📂 Chargement de l'image...")
        self.image_path = image_path
        self.original_image = Image.open(image_path)

        # Image binaire pour l'analyse
        self.gray_image = self.original_image.convert('L')
        self.gray_array = np.array(self.gray_image)

        # Matrice binaire (1=mur, 0=chemin)
        self.maze = self._process_image()
        self.height = len(self.maze)
        self.width = len(self.maze[0])
        print(f"✅ Labyrinthe binarisé: {self.width}x{self.height} pixels")

        # --- Calcul de la transformée en distance ---
        print("🗺️  Calcul de la carte des distances (Distance Transform)...")
        # distance_transform_edt calcule la distance de chaque pixel de chemin (0)
        # au mur (1) le plus proche.
        self.distance_map = distance_transform_edt(self.maze == 0)
        self.max_distance = np.max(self.distance_map)
        print(f"✅ Carte des distances générée (Distance max du centre: {self.max_distance:.1f} pixels)")

        # (Optionnel) Sauvegarder une image de la carte pour déboguer
        try:
            dist_img_array = (self.distance_map / self.max_distance * 255).astype(np.uint8)
            dist_img = Image.fromarray(dist_img_array)
            dist_img.save("debug_distance_map.png")
            print("💾 Carte des distances sauvegardée (debug_distance_map.png)")
        except Exception as e:
            print(f"Erreur sauvegarde debug_distance_map: {e}")
        # --- Fin de la carte des distances ---

        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    def _process_image(self):
        """Convertir l'image en matrice binaire"""
        # < 128 signifie que le noir (valeur 0) devient 1 (mur)
        # et le blanc (valeur 255) devient 0 (chemin)
        maze = (self.gray_array < 128).astype(int)
        return maze

    # [MODIFIÉ]
    def is_valid(self, x, y):
        """Vérifier si la position est valide (pas un mur ET assez loin du bord)"""

        # On force le chemin à être à au moins 2 pixels de tout mur.
        # Augmentez cette valeur si les couloirs sont larges.
        min_clearance = 2

        return (0 <= x < self.width and
                0 <= y < self.height and
                self.distance_map[y][x] > min_clearance)

    def detect_entry_by_border_analysis(self):
        """Détecter l'entrée en analysant les bords"""
        print("\n🔍 Analyse des bords pour détecter l'entrée...")
        border_thickness = 20
        regions = []

        # GAUCHE
        left_region = self.gray_array[:, :border_thickness]; regions.append(('gauche', np.mean(left_region), np.std(left_region)))
        # DROIT
        right_region = self.gray_array[:, -border_thickness:]; regions.append(('droite', np.mean(right_region), np.std(right_region)))
        # HAUT
        top_region = self.gray_array[:border_thickness, :]; regions.append(('haut', np.mean(top_region), np.std(top_region)))
        # BAS
        bottom_region = self.gray_array[-border_thickness:, :]; regions.append(('bas', np.mean(bottom_region), np.std(bottom_region)))

        print("\n📊 Analyse des bords:")
        for side, mean, std in regions:
            print(f"    {side:8} - Moyenne: {mean:.1f}, Écart-type: {std:.1f}")

        max_std_region = max(regions, key=lambda r: r[2])
        max_mean_region = max(regions, key=lambda r: r[1])

        print(f"\n🎯 Bord avec plus de variation: {max_std_region[0]}")
        print(f"🎯 Bord le plus clair: {max_mean_region[0]}")

        candidate_side = max_std_region[0]
        entry = self._find_entry_on_side(candidate_side)

        if entry:
            print(f"✅ ENTRÉE détectée: {entry} sur le bord {candidate_side}")
            return entry

        print(f"⚠️  Tentative sur le bord le plus clair: {max_mean_region[0]}")
        entry = self._find_entry_on_side(max_mean_region[0])

        if entry:
            print(f"✅ ENTRÉE détectée: {entry} sur le bord {max_mean_region[0]}")
            return entry

        print("⚠️  Recherche exhaustive sur tous les bords...")
        return self._find_any_border_passage()

    def _find_entry_on_side(self, side):
        """Trouver un point d'entrée valide sur un côté spécifique"""
        search_depth = 30

        if side == 'gauche':
            for y in range(self.height - 1, -1, -1):
                for x in range(search_depth):
                    if self.is_valid(x, y): return (x, y)
        elif side == 'droite':
            for y in range(self.height):
                for x in range(self.width - 1, self.width - search_depth, -1):
                    if self.is_valid(x, y): return (x, y)
        elif side == 'haut':
            for x in range(self.width):
                for y in range(search_depth):
                    if self.is_valid(x, y): return (x, y)
        elif side == 'bas':
            for x in range(self.width):
                for y in range(self.height - 1, self.height - search_depth, -1):
                    if self.is_valid(x, y): return (x, y)
        return None

    def _find_any_border_passage(self):
        """Recherche exhaustive d'un passage sur n'importe quel bord"""
        search_depth = 40

        # Priorité: gauche-bas
        for y in range(self.height - 1, max(0, self.height - search_depth), -1):
            for x in range(search_depth):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - coin gauche-bas")
                    return (x, y)
        # Bord haut
        for x in range(self.width):
            for y in range(search_depth):
                if self.is_valid(x, y):
                    print(f"✅ Passage trouvé: ({x}, {y}) - bord haut")
                    return (x, y)

        raise Exception("❌ Aucun passage trouvé sur les bords!")

    def find_center(self):
        """Trouver le centre accessible du labyrinthe"""
        print("\n🎯 Recherche du centre du labyrinthe...")
        center_x, center_y = self.width // 2, self.height // 2
        max_radius = max(self.width, self.height) // 2

        for radius in range(0, max_radius, 2):
            for angle in range(0, 360, 15):
                rad = np.radians(angle)
                x = center_x + int(radius * np.cos(rad))
                y = center_y + int(radius * np.sin(rad))

                if self.is_valid(x, y):
                    print(f"✅ Centre trouvé: ({x}, {y})")
                    return (x, y)

        raise Exception("❌ Aucun centre accessible trouvé!")

    def find_entry_and_exit(self):
        """Trouver entrée (bord) et sortie (centre)"""
        print("\n" + "="*60); print("🎯 DÉTECTION AUTOMATIQUE ENTRÉE/SORTIE"); print("="*60)
        entry = self.detect_entry_by_border_analysis()
        exit_center = self.find_center()
        distance = abs(exit_center[0] - entry[0]) + abs(exit_center[1] - entry[1])
        print(f"\n📏 Distance Manhattan: {distance} pixels"); print("="*60)
        return entry, exit_center

    def heuristic(self, point, goal):
        """Distance de Manhattan"""
        return abs(point[0] - goal[0]) + abs(point[1] - goal[1])

    # [MODIFIÉ]
    def a_star(self, start, goal):
        """Algorithme A* qui pénalise les pixels proches des murs"""
        print(f"\n⚡ Application de l'algorithme A* (version 'centrée')...")
        print(f"    Départ (ENTRÉE): {start}")
        print(f"    Arrivée (CENTRE): {goal}")

        start_time = time.time()
        counter = 0
        open_set = []
        heapq.heappush(open_set, (0, counter, start))

        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}
        closed_set = set()
        nodes_explored = 0

        while open_set:
            _, _, current = heapq.heappop(open_set)

            if current == goal:
                end_time = time.time()
                path = self._reconstruct_path(came_from, goal)
                stats = { 'length': len(path), 'time': end_time - start_time, 'nodes_explored': nodes_explored, 'success': True }
                print(f"✅ Chemin trouvé!"); print(f"    📏 Longueur: {len(path)} pas"); print(f"    ⏱  Temps: {stats['time']:.4f} secondes"); print(f"    🔢 Nœuds explorés: {nodes_explored}")
                return path, stats

            if current in closed_set:
                continue

            closed_set.add(current)
            nodes_explored += 1
            x, y = current

            for dx, dy in self.directions:
                neighbor = (x + dx, y + dy)
                nx, ny = neighbor

                # 'is_valid' vérifie maintenant aussi la distance au mur
                if not self.is_valid(nx, ny) or neighbor in closed_set:
                    continue

                # --- Calcul du coût MODIFIÉ ---
                # Le coût est INVERSEMENT proportionnel à la distance au mur
                # Proche du mur (faible distance) -> coût ÉLEVÉ
                # Loin du mur (grande distance) -> coût FAIBLE (coût de 1.0)

                # 'self.distance_map[ny][nx]' = distance du voisin au mur
                # 'self.max_distance' = distance max (centre parfait)

                cost = (self.max_distance - self.distance_map[ny][nx]) + 1.0

                tentative_g = g_score[current] + cost
                # --- Fin de la modification du coût ---

                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, goal)
                    counter += 1
                    heapq.heappush(open_set, (f_score[neighbor], counter, neighbor))

        end_time = time.time()
        stats = { 'length': -1, 'time': end_time - start_time, 'nodes_explored': nodes_explored, 'success': False }
        print(f"❌ Aucun chemin trouvé!")
        return None, stats

    def _reconstruct_path(self, came_from, current):
        """Reconstruire le chemin à partir du dictionnaire 'came_from'"""
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

    def create_animated_gif(self, path, start, goal, output_path='solution.gif',
                            frame_skip=5, duration=50, final_pause=2000):
        """Créer un GIF animé montrant la progression du chemin"""
        print(f"\n🎬 Génération du GIF animé...")
        frames = []
        path_length = len(path)
        frame_indices = list(range(0, path_length, frame_skip))

        if frame_indices[-1] != path_length - 1:
            frame_indices.append(path_length - 1)

        print(f"    📊 {len(frame_indices)} frames à générer...")

        for i, end_idx in enumerate(frame_indices):
            frame_img = self.original_image.convert('RGB')
            draw = ImageDraw.Draw(frame_img)
            current_path = path[:end_idx + 1]

            # Dessiner le chemin rouge
            if len(current_path) > 1:
                draw.line(current_path, fill=(255, 0, 0), width=3)

            # Point de fin (jaune)
            if current_path:
                cx, cy = current_path[-1]; r = 6
                draw.ellipse([(cx-r, cy-r), (cx+r, cy+r)], fill=(255, 255, 0))

            # Point de départ (vert)
            sx, sy = start; r = 8
            draw.ellipse([(sx-r, sy-r), (sx+r, sy+r)], fill=(0, 255, 0))

            # Point d'arrivée (bleu)
            gx, gy = goal; r = 8
            draw.ellipse([(gx-r, gy-r), (gx+r, gy+r)], fill=(0, 0, 255))

            frames.append(frame_img)
            if (i + 1) % 10 == 0:
                print(f"    ⏳ Progression: {i+1}/{len(frame_indices)} frames")

        print(f"    💾 Sauvegarde du GIF...")
        durations = [duration] * (len(frames) - 1) + [final_pause]
        frames[0].save(output_path, save_all=True, append_images=frames[1:],
                       duration=durations, loop=0, optimize=False)

        print(f"✅ GIF animé sauvegardé: {output_path}")
        print(f"    🟢 Vert = ENTRÉE | 🔵 Bleu = SORTIE | 🔴 Jaune = Chemin")

    def visualize_solution(self, path, start, goal, output_path='solution.png'):
        """Créer une image statique avec le chemin tracé"""
        print(f"\n🎨 Génération de l'image statique...")
        result_img = self.original_image.convert('RGB')
        draw = ImageDraw.Draw(result_img)

        if path and len(path) > 1:
            draw.line(path, fill=(255, 0, 0), width=3)

        sx, sy = start; r = 8
        draw.ellipse([(sx-r, sy-r), (sx+r, sy+r)], fill=(0, 255, 0))
        gx, gy = goal; r = 8
        draw.ellipse([(gx-r, gy-r), (gx+r, gy+r)], fill=(0, 0, 255))

        result_img.save(output_path)
        print(f"✅ Image statique sauvegardée: {output_path}")
        return result_img

    def solve_auto(self, output_gif='solution.gif', output_png='solution.png',
                   frame_skip=5, duration=50):
        """Exécute le processus complet de résolution automatique"""
        print("\n" + "="*60); print("🤖 RÉSOLUTION AUTOMATIQUE DU LABYRINTHE"); print("="*60)

        try:
            entry, center = self.find_entry_and_exit()
            path, stats = self.a_star(entry, center)

            if stats['success']:
                self.visualize_solution(path, entry, center, output_png)
                self.create_animated_gif(path, entry, center, output_gif,
                                         frame_skip=frame_skip, duration=duration)

                print("\n" + "="*60); print("✅ RÉSOLUTION RÉUSSIE!"); print("="*60)
                print(f"📊 STATISTIQUES:")
                print(f"    • Longueur du chemin: {stats['length']} pas")
                print(f"    • Temps de calcul: {stats['time']:.4f} secondes")
                print(f"    • Nœuds explorés: {stats['nodes_explored']}")
                print("="*60)
                return path, stats
            else:
                print("\n❌ Impossible de trouver un chemin!")
                return None, stats

        except Exception as e:
            print(f"\n❌ Erreur: {e}")
            traceback.print_exc()
            return None, None


def main():
    """Programme principal"""

    # --- C'EST ICI QUE VOUS METTEZ VOTRE NOM DE FICHIER ---
    image_path = 'maze.jpg' # ou 'imag.png', 'labyrinthe.png', etc.
    # ---------------------------------------------------

    print("🚀 Démarrage du résolveur de labyrinthe...")
    print(f"📁 Fichier: {image_path}\n")

    try:
        solver = MazeAStar(image_path)

        path, stats = solver.solve_auto(
            output_gif='solution_centree.gif',
            output_png='solution_centree.png',
            frame_skip=10, # Augmenter pour un GIF plus rapide (ex: 15)
            duration=40    # Durée de chaque frame en ms
        )

        if path:
            print("\n✨ Fichiers générés:")
            print("    📄 solution_centree.png - Image statique")
            print("    🎬 solution_centree.gif - Animation")
            print("    🗺️  debug_distance_map.png - Visualisation de la carte")

    except FileNotFoundError:
        print(f"\n❌ ERREUR: Le fichier '{image_path}' n'existe pas!")
        print("📝 Placez votre image dans le même dossier que le script")
    except Exception as e:
        print(f"\n❌ ERREUR: {e}")
        traceback.print_exc()


if __name__ == "__main__":
    main()

🚀 Démarrage du résolveur de labyrinthe...
📁 Fichier: maze.jpg

📂 Chargement de l'image...
✅ Labyrinthe binarisé: 426x432 pixels
🗺️  Calcul de la carte des distances (Distance Transform)...
✅ Carte des distances générée (Distance max du centre: 23.0 pixels)
💾 Carte des distances sauvegardée (debug_distance_map.png)

🤖 RÉSOLUTION AUTOMATIQUE DU LABYRINTHE

🎯 DÉTECTION AUTOMATIQUE ENTRÉE/SORTIE

🔍 Analyse des bords pour détecter l'entrée...

📊 Analyse des bords:
    gauche   - Moyenne: 185.8, Écart-type: 113.1
    droite   - Moyenne: 161.0, Écart-type: 122.7
    haut     - Moyenne: 173.7, Écart-type: 118.6
    bas      - Moyenne: 158.7, Écart-type: 123.3

🎯 Bord avec plus de variation: bas
🎯 Bord le plus clair: gauche
✅ ENTRÉE détectée: (7, 422) sur le bord bas

🎯 Recherche du centre du labyrinthe...
✅ Centre trouvé: (213, 216)

📏 Distance Manhattan: 412 pixels

⚡ Application de l'algorithme A* (version 'centrée')...
    Départ (ENTRÉE): (7, 422)
    Arrivée (CENTRE): (213, 216)
✅ Chemin 